In [ ]:
from mmdet.apis import DetInferencer
import os
import numpy as np
from PIL import Image

def crop_image(filename, x, y, w, h, out_dir='/opt/imagry/Downloads/crops'):
    basename = os.path.basename(filename)
    prefix, ext = os.path.splitext(basename)
    img = np.array(Image.open(filename))
    cropped_img = img[y:y+h, x:x+w, :]
    cropped_img = Image.fromarray(cropped_img)
    os.makedirs(out_dir, exist_ok=True)
    out_path = os.path.join(out_dir, f'{prefix}_{x}_{y}_{w}_{h}{ext}')
    print(out_path)
    cropped_img.save(out_path)
    return out_path

In [ ]:
# models is a list of model names, and them will print automatically
models = DetInferencer.list_models('mmdet')

In [ ]:

from mmdet.evaluation import CocoMetric
from mmdet.apis import DetInferencer
inferencer = DetInferencer(model='rtmdet_tiny_8xb32-300e_coco')
import torch
dataset_meta = inferencer.model.__dict__['dataset_meta']
def process_result(result, to_tensor=True):
    result = result['predictions'][0]
    if to_tensor:
        for k in ['bboxes', 'scores', 'labels']:
            result[k] = torch.tensor(result[k])
    return result

def processed_result_to_annotation(result):
    anns = []
    for i in range(len(result['labels'])):
        if not isinstance(result['bboxes'][i], list):
            bbox = list((result['bboxes'][i].cpu().numpy()))
        else:
            bbox = result['bboxes'][i]
                    
        anns.append(dict(bbox_label=result['labels'][i], bbox=bbox))
    return anns
    
def get_results(model, img_paths, out_dir, device='cuda:0'):
    inferencer = DetInferencer(model, device=device)
    results = []
    for img_path in img_paths:
        results.append(inferencer(img_path, out_dir=out_dir))
    del inferencer
    torch.cuda.empty_cache()
    return results
    
def compute_confusion(model_a, model_b, img_paths, out_dir_a, out_dir_b, metric='coco/bbox_mAP'):
    confusion = []
    results_a = get_results(model_a, img_paths, out_dir_a)
    results_b = get_results(model_b, img_paths, out_dir_b)

    for i in range(len(results_a)):
        results_a_ = process_result(results_a[i])
        results_b_ = process_result(results_b[i], to_tensor=False)
        coco_metric = CocoMetric(ann_file= None, metric=['bbox'], classwise=True, outfile_prefix='/opt/imagry/Downloads/confusion')
        coco_metric.dataset_meta = dataset_meta
        coco_metric.process({}, [
            dict(
                pred_instances=results_a_,
                img_id=0,
                ori_shape=(1920, 1080),
                instances=processed_result_to_annotation(results_b_))
        ])
        eval_results = coco_metric.evaluate(size=1)
        confusion.append((img_paths[i], eval_results[metric]))
    return confusion, eval_results

In [ ]:
import os
img_folder = '/home/imagry/Downloads/2024-03-12T09_16_53/3d_images/0/left'
img_paths = os.listdir(img_folder)
img_paths = [os.path.join(img_folder, p) for p in img_paths]
confusion, eval_results = compute_confusion('mask-rcnn_r50_fpn_dconv_c3-c5_1x_coco', 'mask-rcnn_r101-dconv-c3-c5_fpn_1x_coco', img_paths, 'outputs_a/', 'outputs_b/')

In [ ]:
from PIL import Image
import numpy as np

def visualize(confusion, folder, folder_a, folder_b, case, output_folder='/opt/imagry/Downloads/confusion'):
    os.makedirs(os.path.join(output_folder, case), exist_ok=True)
    for fn, c in confusion:
        print(fn, c)
        basename = os.path.basename(fn)
        im = Image.open(os.path.join(folder, basename))
        im_a = Image.open(os.path.join(folder_a, basename))
        im_b = Image.open(os.path.join(folder_b, basename))

        Image.fromarray(np.hstack((np.array(im), np.array(im_a), np.array(im_b)))).save(os.path.join(output_folder, case, basename))      
N=100

confusion.sort(key = lambda x: x[1])

best_images = confusion[-N:]
worst_images = confusion[:N]

print('.....Best Images......')
visualize(best_images, img_folder, './outputs_a/vis', './outputs_b/vis', 'best')

print('.....Worst Images......')
visualize(worst_images, img_folder, './outputs_a/vis', './outputs_b/vis', 'worst')